In [8]:
### This script handles the assignment of our TAs to available shifts.
#As input it takes a matrix where each row represents a single TA and each column represents
#one shift. This matrix will be referred to as the "availability matrix"
#There are generally 3 TAs or more on a given shift.

#The availability matrix is stored in a google sheets file, which is shared with a service 
#account, the credentials to which are contained in "client.json"

#The script imports the availabilty matrix through the use of the service account, and then
#plans a roster in a way that tries to give equally many shifts to all workers.
#This is done with a Metropolis-Hastings inspired algorithm i thought up, which is probably
#highly inefficient, but it gets the job done.

#Once the roster is finished, the same service account is used to upload it to a seperate
#google sheets file, containing all the rosters from that year.

import numpy as np
import random
import gspread

json = "client.json"


totalname = "Vagtplan 2021/2022"
signupname = "Signup 2021/2022"
sheetname = "Blok 4"
cakethisblock = True #One TA should do the assignment of the week, while another should bring
#cake. During covid the homework help was provided over zoom, and so the cake didnt matter
#and an option to disable cake was added.

poed = 3 #poed = people on each day. The amount of people that are required per shift for this
#course

hourspershift = 3 #The amount of hours each shift nets you on your paycheck.


account = gspread.service_account(filename = "client.json")
totalfile = account.open(totalname)
signupfile = account.open(signupname)

mainasarray = np.array(totalfile.worksheet("main").get_all_values())
cakeasarray = np.array(totalfile.worksheet("kage").get_all_values())
signupasarray = np.array(signupfile.worksheet(sheetname).get_all_values())

#Function to find the first all-zero row or column in an array
def findzero(arr, dim = 1):
    if arr.size == 0:
        return None
    
    if dim == 0:
        slicer = lambda i: (i, slice(None))
    elif dim == 1:
        slicer = lambda i: (slice(None), i)
    try:
        return np.nonzero([(arr[slicer(i)] == "").all() for i in range(arr.shape[dim])])[0][0]
    except:
        return None


weeks = findzero(signupasarray, 1) - 1

In [9]:
mask = np.nonzero(mainasarray[:,0])
names = mainasarray[:, 0][mask]
prevhours = mainasarray[:, -1][mask].astype(int)

#crop the array to get only the relevant data
availability = signupasarray[1:,1:findzero(signupasarray, 1)]
availability = np.char.lower(availability)
prevcake = cakeasarray[1:findzero(cakeasarray, 0), -1].astype(int)*2

numpeople = len(names)
numdays = availability.shape[1]

#number of additional hours for this block that need to be assigned
addhours = (poed+1) * weeks * hourspershift

target = np.mean(prevhours) + addhours/numpeople
targetcake = np.mean(prevcake) + numdays/numpeople * 2

thething = np.zeros((poed,numdays), dtype = mainasarray.dtype)

index = np.arange(numdays)
random.shuffle(index) #random.shuffle() is inplace

def countscore():
    global newhours, oldscore, newscore, oldhours, cake
    oldhours = np.copy(newhours)
    oldscore = newscore
    for i in range(numpeople):
        cake[i] = np.count_nonzero(thething[1,:] == names[i])
        newhours[i] = np.count_nonzero(thething == names[i]) + np.count_nonzero(thething[0,:] == names[i])
    newhours = newhours * 3
    cake = cake * 2
    newscore = round(((prevhours+newhours-target)**2).sum() + ((prevcake+cake-targetcake)**2).sum(), 4)


def init():
    global thething, cake, newhours, oldthing, newscore
    for i in index:
        whocan = names[(availability[:,i] == 'x')]
        random.shuffle(whocan)
        thething[0,i] = whocan[0]
        whocan = names[np.logical_or(availability[:,i] == 'x', availability[:,i] == 'v')]
        whocan = whocan[whocan != thething[0,i]]
        random.shuffle(whocan)
        thething[1:,i] = whocan[:poed-1]
    newhours = np.zeros(numpeople)
    cake = np.zeros(numpeople)
    newscore = float("inf")
    countscore()
    oldthing = np.copy(thething)





def checker(i, j):
    global thething
    if i == 0:
        whocan = names[(availability[:,j] == 'x')]
        whocan = np.setdiff1d(whocan, thething[:,j])
        if whocan.size>0:
            random.shuffle(whocan)
            thething[i,j] = whocan[0]
    else:
        whocan = names[(availability[:,j] == 'x') + (availability[:,j] == 'v')]
        whocan = np.setdiff1d(whocan,thething[:,j])
        if whocan.size>0:
            random.shuffle(whocan)
            thething[i,j] = whocan[0]


def letsrunthis(iter):
    global bestthing, bestscore, thething, newscore, oldscore, oldthing, besthours, bestcake
    for k in range(iter):
        for i in range(poed):
            for j in range(numdays):
                checker(i,j)
                countscore()
                if newscore <= oldscore or random.uniform(0,1)<0.02:
                    oldthing = np.copy(thething)
                    if newscore <= bestscore:
                        bestthing = np.copy(thething)
                        bestscore = newscore
                        besthours = np.copy(newhours)
                        bestcake = np.copy(cake)
                else:
                    thething = np.copy(oldthing)
                    newscore = oldscore
        if k % 50 == 0:
            print(newscore, bestscore)


bestscore = float("inf")



In [10]:
for i in range(10):
    init()
    letsrunthis(2*10**2)

186.375 186.375
176.375 152.375
188.375 152.375
160.375 152.375
258.375 152.375
186.375 152.375
170.375 152.375
170.375 152.375
228.375 152.375
170.375 152.375
238.375 152.375
152.375 152.375
188.375 152.375
152.375 152.375
152.375 152.375
194.375 152.375
196.375 152.375
170.375 152.375
160.375 152.375
178.375 152.375
186.375 152.375
212.375 152.375
178.375 152.375
170.375 152.375
202.375 152.375
188.375 152.375
186.375 152.375
186.375 152.375
238.375 152.375
188.375 152.375
170.375 152.375
152.375 152.375
222.375 152.375
152.375 152.375
170.375 152.375
178.375 152.375
186.375 152.375
188.375 152.375
152.375 152.375
186.375 152.375


In [11]:
outsheet = totalfile.worksheet(sheetname)
outsheet.update_cell(1, 1, sheetname)
outsheet.update_cell(1, 2, signupasarray[0, 0])

for ind, date in zip(range(1,weeks+1), signupasarray[0, 1:weeks+1]):
    outsheet.update_cell(2, ind, date)
    
a1notationrow = lambda row: gspread.utils.rowcol_to_a1(row, 1) + ":" + gspread.utils.rowcol_to_a1(row, weeks)

outsheet.format(a1notationrow(2), {'textFormat': {'bold': True}})

outsheet.update("A3", bestthing.tolist())
outsheet.update_cell(3, weeks+2, "Opgave")

colorblue = lambda rang: outsheet.format(rang, {"backgroundColor": {
      "red": 0.6431373,
      "green": 0.7607843,
      "blue": 0.95686275
    }})

colorblue(gspread.utils.rowcol_to_a1(3, weeks+2))
colorblue(a1notationrow(3))


if cakethisblock:
    outsheet.update_cell(4, weeks+2, "Kage")
    colorgreen = lambda rang: outsheet.format(rang, {"backgroundColor": {
          "green": 1,
        }})
    
    colorgreen(gspread.utils.rowcol_to_a1(4, weeks+2))
    colorgreen(a1notationrow(4))


In [12]:
bestthing

array([['Anna', 'Josefine', 'Emil', 'Freja', 'Clara', 'Andreas'],
       ['Emil', 'Anna', 'Freja', 'Oline', 'Josefine', 'Clara'],
       ['Freja', 'Andreas', 'Clara', 'Josefine', 'Emil', 'Oline']],
      dtype='<U28')

In [13]:
for i in range(len(names)):
    print(names[i].replace(" ",""), "    \t", (besthours+prevhours)[i],"\t", ((bestcake + prevcake)/2)[i])

print("\n")
for i in bestthing:
    print(i)
print("\n",bestscore)

Andreas     	 54.0 	 4.0
Anna     	 54.0 	 5.0
Clara     	 54.0 	 4.0
Emil     	 54.0 	 5.0
Freja     	 54.0 	 4.0
Oline     	 51.0 	 5.0
Josefine     	 54.0 	 4.0
Nicolas     	 42.0 	 2.0


['Anna' 'Josefine' 'Emil' 'Freja' 'Clara' 'Andreas']
['Emil' 'Anna' 'Freja' 'Oline' 'Josefine' 'Clara']
['Freja' 'Andreas' 'Clara' 'Josefine' 'Emil' 'Oline']

 152.375


In [17]:
# np.savetxt("Its done.csv",bestthing,delimiter=";", fmt = "%s")